In [ ]:
import elasticsearch
import git
import os
from git import Repo
import sys

def create_components(git_url,esurl,localdir):
    # if no url supplied for Elastic, assume the localhost
    if esurl =='':
        try:
            es = elasticsearch.Elasticsearch(['http://localhost:9200/'], maxsize=500, block=False)
        except:
            print('Elasticsearch not running at localhost:9200')
            sys.exit(1)
    else:
        try:
            es = elasticsearch.Elasticsearch([esurl], maxsize=500, block=False)
        except:
            print('Elasticsearch not running at the given URL. For default localhost, do not provide the argument')
            sys.exit(1)
        
    # Get the default commit index name
    es_index_raw = str.split(git_url,'/')[-1].split('.')[0]+'_'+'index'
    # Get the default blame index name
    es_blame_index_raw = str.split(git_url,'/')[-1].split('.')[0]+'_'+'blame'+'_'+'index'
    es_index = es_index_raw.lower()
    es_blame_index = es_blame_index_raw.lower()
    # Create elasticsearch instance
    
    # If local Repo path is not supplied, create default path in '/tmp'
    if localdir == '':
        if sys.platform == 'linux':
            local_dir ='/tmp/'+str.split(git_url,'/')[-1].split('.')[0]
            clone_dir = '/tmp'
        else:
            local_dir ='C:\\Downloads'+str.split(git_url,'/')[-1].split('.')[0]
            clone_dir = 'C:\\Downloads'
    else:
        local_dir = localdir+'/'+str.split(git_url,'/')[-1].split('.')[0]
        clone_dir = localdir
    # Check if the local Repo already exists
    if os.path.isdir(local_dir):
        # Load the local Repo
        try:
            repo = git.Repo(local_dir)   
        # Get the latest commit object in the local Repo
            local_commit = repo.commit()     
        except:
            print('No valid Repo found at the location. If unsure, remove the directory and try without local dir argument')
            sys.exit(1)
                       # latest local commit 
        
        # Get the latest commit object in the remote Repo
        remote = git.remote.Remote(repo, 'origin')      # remote repo
        info = remote.fetch()[0]                        # fetch changes
        remote_commit = info.commit  
        
        # If latest commit in local and remote differ then refresh the local Repo
        if (local_commit.hexsha == remote_commit.hexsha ):
            print('No changes in the Repo...')
            # kc - but still run it to force processing - remove below line when we move this to production
            #store_commit_data(local_dir,es,es_index,es_blame_index,local_commit.hexsha,remote_commit.hexsha )
        else:    
            repo = git.Repo(local_dir) 
            o = repo.remotes.origin
            o.pull()
            # Analyse and store additional commit data
            store_commit_data(local_dir,es,es_index,es_blame_index,local_commit.hexsha,remote_commit.hexsha )
    else:
        # If no local Repo exists, clone the Repo
        try:
            if sys.platform == 'linux':
                git.Git(clone_dir).clone(git_url)
            else:
                git.Git('C:\\Downloads').clone(git_url)
        except:
            print('Not able to clone the Repo. If there is a non Git directory with the  same name, delete it and then try')
            sys.exit(1)
        # Delete the elastic indices, if exist
        es.indices.delete(index=es_index, ignore=[400, 404])
        es.indices.delete(index=es_blame_index, ignore=[400, 404])
        # Create new elastic indices
        es.indices.create(es_index)
        es.indices.create(es_blame_index)
        # Call the function to store the necessary commit data
        store_commit_data(local_dir,es,es_index,es_blame_index,'None','None')

    return es,es_index,es_blame_index

In [ ]:
# Insert records to Elastic
# This is useless for now but we can decide to use it along with
#        elastic connection pools. 
#        We can avoid bulk import with this function

import elasticsearch
from elasticsearch_dsl import Search,Q

def store_to_elastic(elastic_object, index_name, document_type, record):
    try:
        outcome = elastic_object.index(index=index_name, doc_type =document_type, body=record)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))

In [ ]:
# Function to process individual modification(mod) in commit

import re

def process_mod(commit, q_doclist, q_blamelist, local_dir, repo, mod):
    
            # Creating empty lists for carrying commit data
            doclist = []
            blamelist = []
            #print('In Process Mod')
            
            #Building Commit tuples
            commit_data = {'hash':commit.hash,'Author':commit.author.name,'Email':commit.author.email,
                               'message':commit.msg,'authored_date':commit.author_date,
                               'Committer':commit.committer.name,'committed_date':commit.committer_date,
                               'no._of_branches':len(commit.branches),'merge_commit?':commit.merge,
                               'no._of_mod_files':len(commit.modifications),'dmm_unit_size':commit.dmm_unit_size,
                               'dmm_unit_complexity':commit.dmm_unit_complexity,'dmm_unit_interfacing':commit.dmm_unit_interfacing,
                               'file_name':mod.filename, 'file_path':mod.new_path, 'complexity': mod.complexity, 'functions': len(mod.methods),
                               'lines_added':mod.added,'lines_removed': mod.removed,'loc':mod.nloc,'size': 0 if mod.source_code is None else len(mod.source_code.splitlines()),'tokens':mod.token_count
                                       
                                        }
            # loading each commit tuple into the list
            doclist.append(commit_data)
            #print("doclistlength: "+str(len(doclist)))
            # Each modification object contains diff_parsed tag with "added" and "deleted" properties. 
            alines = mod.diff_parsed['added']
            # "added" property is a tuple list with line number and actual line text. List of text lines is extracted
            addedlines = [x[1] if len(alines)>0 else 'None' for x in alines ]
                    
            # Each modification object contains diff_parsed tag with "added" and "deleted" properties. 
            blines = mod.diff_parsed['deleted']
            # "deleted" property is a tuple list with line number and actual line text. List of text lines is extracted
            deletedlines = [x[1] if len(blines)>0 else 'None' for x in blines]
            count = 0
                    
            # Absolute path of the file in the cloned repo. This is required to validate that the file has not been deleted in the subsequent commits
            newfilepath = local_dir+'/'+str(mod.new_path)
            # For bug fix commits, retrieving the blame data. Using Regex on Commit messages to identify bug fix commits
            if len(re.findall(r"\bbug\b|\bBug\b|\bFix\b|\bfix\b",commit.msg))>0 and os.path.isfile(newfilepath):# & len(addedlines)>0:
                # Running Git Blame on each added line.
                for eachline in addedlines:
                    repo_blame = repo.blame(commit.hash,mod.new_path,eachline)
                    # Git Blame of a line can produce multiple records with each record representing a past modification
                    for blame_record in repo_blame:
                        # Git Blame produces duplicate records (Don't know why). Attempt to ignore duplicated by comparting the current record with the previous record
                        # Also Git Blame produces record of the same commit hash, which can be ignored
                        #prev_record = ''
                        if str(commit.hash) !=str(blame_record[0]): #and (str(blame_record[0]) != prev_record):
                        # Building Blame tuple for each Blame record
                            blame_doc = {'orig_hash':commit.hash,'blame_hash':str(blame_record[0]),
                                            'file':mod.new_path}    
                            # Loading blame data into the list
                            blamelist.append(blame_doc)
                            #print("blamelistlength: "+str(len(blamelist)))
                            #prev_record = blame_record[0]

            # Send these to 'process_commit'
            
            #q_doclist.put(doclist)
            
            q_blamelist.put(blamelist)
           
            # delete our lists
            del doclist
            del blamelist
            del blamelist_fil

In [ ]:
def process_blame(commit, q_blamelist,eachline, local_dir, repo, npath):
    blamelist = []
    repo_blame = repo.blame(commit.hash,npath,eachline)
                    # Git Blame of a line can produce multiple records with each record representing a past modification
    for blame_record in repo_blame:
                        # Git Blame produces duplicate records (Don't know why). Attempt to ignore duplicated by comparting the current record with the previous record
                        # Also Git Blame produces record of the same commit hash, which can be ignored
                        #prev_record = ''
        if str(commit.hash) !=str(blame_record[0]): #and (str(blame_record[0]) != prev_record):
                        # Building Blame tuple for each Blame record
            blame_doc = {'orig_hash':commit.hash,'blame_hash':str(blame_record[0]),
                                            'file':npath}    
                            # Loading blame data into the list
            blamelist.append(blame_doc)
                            #print("blamelistlength: "+str(len(blamelist)))
                     #prev_record = blame_record[0]
    q_blamelist.put(blamelist)

In [ ]:
# Function to processes each commit

import concurrent.futures
import elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search,Q
import queue
import pandas as pd

def process_commit(commit, repo, es, es_index, es_blame_index, local_dir):

    # Creating empty lists for carrying commit data
    doclist = []
    blamelist = []
    
    # Create queues for commit data and blame data
    #q_doclist = queue.Queue()
    q_blamelist = queue.Queue()
    for mod in commit.modifications:
        commit_data = {'hash':commit.hash,'Author':commit.author.name,'Email':commit.author.email,
                               'message':commit.msg,'authored_date':commit.author_date,
                               'Committer':commit.committer.name,'committed_date':commit.committer_date,
                               'no._of_branches':len(commit.branches),'merge_commit?':commit.merge,
                               'no._of_mod_files':len(commit.modifications),'dmm_unit_size':commit.dmm_unit_size,
                               'dmm_unit_complexity':commit.dmm_unit_complexity,'dmm_unit_interfacing':commit.dmm_unit_interfacing,
                               'file_name':mod.filename, 'file_path':mod.new_path, 'complexity': mod.complexity, 'functions': len(mod.methods),
                               'lines_added':mod.added,'lines_removed': mod.removed,'loc':mod.nloc,'size': 0 if mod.source_code is None else len(mod.source_code.splitlines()),'tokens':mod.token_count
                                       
                                        }
            # loading each commit tuple into the list
        doclist.append(commit_data)
            #print("doclistlength: "+str(len(doclist)))
            # Each modification object contains diff_parsed tag with "added" and "deleted" properties. 
        alines = mod.diff_parsed['added']
            # "added" property is a tuple list with line number and actual line text. List of text lines is extracted
        addedlines = [x[1] if len(alines)>0 else 'None' for x in alines ]
                    
            # Each modification object contains diff_parsed tag with "added" and "deleted" properties. 
        blines = mod.diff_parsed['deleted']
            # "deleted" property is a tuple list with line number and actual line text. List of text lines is extracted
        deletedlines = [x[1] if len(blines)>0 else 'None' for x in blines]
        no_of_mod_files = len(commit.modifications)
        lines_added = mod.added
        count = 0
                    
            # Absolute path of the file in the cloned repo. This is required to validate that the file has not been deleted in the subsequent commits
        newfilepath = local_dir+'/'+str(mod.new_path)
            # For bug fix commits, retrieving the blame data. Using Regex on Commit messages to identify bug fix commits
    # Have lightweight threads process each modification(mod)
    # Remember that each modification is a single line in the commit and we can have many
    # Since most of the work is I/O bound, processors should be mostly idle, 
    #       we can load up with a LOT of threads - set he max threads to 200 
        if len(re.findall(r"\bbug\b|\bBug\b|\bFix\b|\bfix\b",commit.msg))>0 and os.path.isfile(newfilepath) and no_of_mod_files <15 and lines_added < 1000:# and no_of_mod_files <15 and lines_added < 1000:# & len(addedlines)>0:
            with concurrent.futures.ThreadPoolExecutor(max_workers=200) as executor:
                futures = [executor.submit(process_blame, commit, q_blamelist,eachline, local_dir, repo, newfilepath) for eachline in addedlines]

        # Wait for all threads to complete
            executor.shutdown(wait=True)


     
    # Vamsi - Changed the method to assert the empty queue
    while not q_blamelist.empty():
    #while (q_blamelist.empty == False):
        blamelist.append(q_blamelist.get())    

    
    # Each thread produces a list of tuples (in this case list of one tuple)
    # When data from queue is appended to a list, it produces list of lists instead of list of tuples
    # It is required to flatten the list of lists into list of tuples. Bulkloading to elastic won't work otherwise
    #doclist = [item for sublist in doclist for item in sublist]
    
    
    # Each thread produces a list of tuples (in this case list of one tuple)
    # When data from queue is appended to a list, it produces list of lists instead of list of tuples
    # It is required to flatten the list of lists into list of tuples. Bulkloading to elastic won't work otherwise
    blamelist = [item for sublist in blamelist for item in sublist]
    #doclist = doclist[0]
    # using elasticsearch.py's helper tools to bulk load into elasticsearch's commit index           
    helpers.bulk(es,doclist,index=es_index,doc_type ='commit_data',request_timeout = 2000)
    # Since Git Blame produces duplicate data, getting only unique records
    #blamelist_fil = [i for n, i in enumerate(blamelist) if i not in blamelist[n + 1:]]
    blame_df = pd.DataFrame(blamelist)
    #print(blame_df.shape)
    
    blame_df_clean = blame_df.drop_duplicates()
    #print(blame_df_clean.shape)
    # using elasticsearch.py's helper tools to bulk load into elasticsearch's blame index
    #print('blamelist_fil: '+str(len(blamelist_fil)))
    #helpers.bulk(es,blamelist_fil,index=es_blame_index,doc_type ='blame',request_timeout = 2000)
    blamelist_fil = []
    df_iter = blame_df_clean.iterrows()
    for index, document in df_iter:
        blamelist_fil.append(document.to_dict())
    #print(blame_df_clean.shape)        
    #print(len(blamelist_fil))
    #helpers.bulk(es, doc_generator(blame_df_clean,es_blame_index))
    # using elasticsearch.py's helper tools to bulk load into elasticsearch's blame index
    helpers.bulk(es,blamelist_fil,index=es_blame_index,doc_type ='blame',request_timeout = 2000)
    # delete our lists
    del doclist
    del blamelist
    del blamelist_fil

In [ ]:
# Function to analyse all commits (and import to elastic)

import elasticsearch
from elasticsearch_dsl import Search,Q
from elasticsearch import helpers
import pandas as pd
from pydriller import RepositoryMining
import git
from git import Repo
import os
import multiprocessing as mp
from datetime import datetime

def store_commit_data(local_dir,es,es_index,es_blame_index,local_commit,remote_commit):

    store_start = datetime.now()
    print('starting store_commit', store_start)
    
    repo = Repo(local_dir)

    # Create Multithreading pool to use full CPU
    pool = mp.Pool(mp.cpu_count())
    
    # If the Repo has just been cloned, the program will traverse the whole Repo
    if(local_commit == 'None'):
        [pool.apply_async(process_commit(commit, repo, es,es_index,es_blame_index, local_dir)) for commit in RepositoryMining(local_dir).traverse_commits()]
        
    else:
        [pool.apply_async(process_commit(commit, repo, es,es_index,es_blame_index, local_dir)) for commit in RepositoryMining(local_dir,from_commit = local_commit, to_commit = remote_commit).traverse_commits()]

    # Close Multiprocessing pool
    pool.close()
    pool.join()
    store_end = datetime.now()
    print('exiting store_commit', store_end)
    print('time taken by store_commit', (store_end - store_start))
    
    # Very important to explicitly refresh the Elastic indices as they are not automatically done.
    es.indices.refresh([es_blame_index,es_index])

In [ ]:
# Realised that git blame and most of our code is heavily constrained by disk speed. 
#       We can create and use an in memory file system. In Linux we can use /tmp 
from pathlib import Path

#Set tmpfs if you have tmpfs. Else leave this as '/tmp' /tmp is usually in RAM 
tmpfs = '/tmp'

if (tmpfs == ''):
    Home_address = str(Path.home())
else:
    #Set Home_address to the tmpfs 
    Home_address = tmpfs
 
print(Home_address)
localdir = Home_address + '/cg_Repos'
print(localdir)

In [ ]:
# Calling the base method to process and load the commit data. See "create_components" method for detail
#localdir = '/home/vamsi'
#p1 = create_components('https://github.com/R-Knowsys/elasticray.git','','')
p2 = create_components('https://github.com/chaoss/grimoirelab.git','','')
#p3 = create_components('https://github.com/microsoft/vscode.git','',localdir)
#p4 = create_components('https://github.com/tensorflow/tensorflow.git','',localdir)    
#p5 = create_components('https://github.com/facebook/react-native.git','',localdir)
#p6 = create_components('https://github.com/kcramakrishna/Digital_Assistant_Client.git','',localdir)


In [ ]:
import elasticsearch
from elasticsearch_dsl import Search,Q
import pandas as pd

def get_user_data(es_instance,commit_index,blame_index):
        # Assigning Elastic instance, Commit Elastic Index and Blame Elastic Index to variables
        es = es_instance
        es_ma_index = commit_index
        es_bl_index = blame_index
        # Using Elasticsearch DSL function to get the data of Commit index
        blame_es_data = Search(using=es, index=es_bl_index)
        # Loading data into a dictionary
        blame_dict = [hit.to_dict() for hit in blame_es_data.scan()]
        # Using Elasticsearch DSL function to get the data of Blame index
        commit_es_data = Search(using=es, index=es_ma_index)
        # Loading data into a dictionary
        commit_dict = [hit.to_dict() for hit in commit_es_data.scan()]
        # Creating pandas dataframe for commit data
        commit_frame = pd.DataFrame(commit_dict)
        # Creating pandas dataframe for blame data
        blame_frame = pd.DataFrame(blame_dict)
        blame_frame['file'] = blame_frame['file'].apply(lambda x:x.split('/')[-1])
        # Getting the blame row count. If the frmae is empty, it means all the records are clean
        blame_count = blame_frame.shape[0]
        
        if blame_count>0:
            # Adding a column to Blame frame indicating that the row represents a Buggy commit
            blame_frame['type'] = 'Buggy'
            # Combining Commit frmae with Blame frame. An additional column called 'type' gets added to the Commit frame.
            comb_frame = pd.merge(commit_frame,blame_frame,how='left',left_on = ['hash','file_name'],right_on = ['blame_hash','file'])
        else:
            # If the Blame frame is empty, no need to merge.
            comb_frame=commit_frame
        # When merging happnes and 'type' column gets added to the main Commit frame, The rows that are not part of Blame frame are filled with 'Nan'.
        # Here, all the NaNs fro 'type' column are replaced with 'Clean' label.
        # Effectively, Each commit file (one Commit can contain more than one file) is categorised as either Buggy or Clean.
        comb_frame['type'] = comb_frame['type'].fillna('Clean')
        
        # Cleaning and retaining the required columns
        comb_frame_refined = comb_frame[['hash', 'Author','Committer', 'Email', 'message',                               'committed_date', 'no._of_branches', 'merge_commit?',
                                        'no._of_mod_files', 'dmm_unit_size', 'dmm_unit_complexity','dmm_unit_interfacing',
                                        'file_path', 'complexity','functions', 'lines_added', 'lines_removed', 'size', 'tokens',
                                        'type']]
        # Commit hash raw value is very long. Cutting the value into first ten chars 
        # Assumption is that the first ten chars rednder necessary uniqueness. May need to revisit later
        #comb_frame_refined['hash'] = comb_frame_refined['hash'].str.slice(0,10)

        # Changing the type from string to Data. Used Pacific time zone. Heard the pacific coast is beautiful
        commit_frame['committed_date'] = commit_frame['committed_date'].astype('str').apply(lambda x: pd.to_datetime(x).tz_convert('US/Pacific'))
        
        # Sorting the frame by committes date
        comb_frame_refined = comb_frame_refined.sort_values('committed_date', ascending=False)
        
        #------ Attempting to score developers with a simple/simplistic calculation -----/
        
        # Replacing all NaN fields with zero
        comb_frame_refined = comb_frame_refined.fillna(0)
        # Adding commit count (actually commit file count) for each other column
        comb_frame_refined['commit_count'] = comb_frame_refined.groupby('Author').hash.transform('nunique')
        # Adding up three dmm values creating a new column
        comb_frame_refined['riskfreescore'] = comb_frame_refined['dmm_unit_size']+comb_frame_refined['dmm_unit_complexity']+comb_frame_refined['dmm_unit_interfacing']
        # Adding up "Lines Added" and "Lines Removed" and creating a new column
        comb_frame_refined['lineschanged'] = comb_frame_refined['lines_added']+comb_frame_refined['lines_removed']
        # Simple calculation to come up with a score for each row
        comb_frame_refined['score'] = comb_frame_refined.apply(lambda x:((x.riskfreescore)*10+(x.lineschanged)+(x.complexity)*10)/10,axis =1)
        # Filtering out the necessary columns and creating a new frame
        dev_score = comb_frame_refined[['Author','commit_count','score','type']]
        # Score is scaled based on the type of commit file
        dev_score['scaled_score'] = dev_score.apply(lambda x: x.score if x.type == 'Clean' else x.score/2,axis=1)
        # Retaining the necessary columns
        dev_score_refined = dev_score[['Author','commit_count','scaled_score']]
        # Calculating the Average score for the Author. A simple arithmetic mean.
        dev_score_refined['average_score'] = dev_score_refined.groupby('Author').scaled_score.transform('mean').round()
        dev_score_final = dev_score_refined[['Author','commit_count','average_score']].drop_duplicates().sort_values('average_score', ascending=False).reset_index().drop(columns = 'index') 
        return dev_score_final

In [ ]:
get_user_data(p1[0],p1[1],p1[2])

In [ ]:
import elasticsearch
from elasticsearch_dsl import Search,Q
import pandas as pd

def get_latest_commits(es_instance,commit_index,blame_index):
        # Assigning Elastic instance, Commit Elastic Index and Blame Elastic Index to variables
        es = es_instance
        es_ma_index = commit_index
        es_bl_index = blame_index
        # Using Elasticsearch DSL function to get the data of Commit index
        blame_es_data = Search(using=es, index=es_bl_index)
        # Loading data into a dictionary
        blame_dict = [hit.to_dict() for hit in blame_es_data.scan()]
        # Using Elasticsearch DSL function to get the data of Blame index
        commit_es_data = Search(using=es, index=es_ma_index)
        # Loading data into a dictionary
        commit_dict = [hit.to_dict() for hit in commit_es_data.scan()]
        # Creating pandas dataframe for commit data
        commit_frame = pd.DataFrame(commit_dict)
        # Creating pandas dataframe for blame data
        blame_frame = pd.DataFrame(blame_dict)
        blame_frame['file'] = blame_frame['file'].apply(lambda x:x.split('/')[-1])
        # Getting the blame row count. If the frmae is empty, it means all the records are clean
        blame_count = blame_frame.shape[0]
        #print(blame_frame.columns)
        
        if blame_count>0:
            # Adding a column to Blame frame indicating that the row represents a Buggy commit
            blame_frame['type'] = 'Buggy'
            # Combining Commit frmae with Blame frame. An additional column called 'type' gets added to the Commit frame.
            #comb_frame = pd.merge(commit_frame,blame_frame,how='left',left_on = ['hash','file_path'],right_on = ['blame_hash','file'])
            comb_frame = pd.merge(commit_frame,blame_frame,how='left',left_on = ['hash','file_name'],right_on = ['blame_hash','file'])
            
            
        else:
            # If the Blame frame is empty, no need to merge.
            comb_frame=commit_frame
        # When merging happnes and 'type' column gets added to the main Commit frame, The rows that are not part of Blame frame are filled with 'Nan'.
        # Here, all the NaNs fro 'type' column are replaced with 'Clean' label.
        # Effectively, Each commit file (one Commit can contain more than one file) is categorised as either Buggy or Clean.
        print(comb_frame[comb_frame['type']=='Buggy'].shape)
        comb_frame['type'] = comb_frame['type'].fillna('Clean')
        
        # Cleaning and retaining the required columns
        comb_frame_refined = comb_frame[['hash', 'Author','Committer', 'Email', 'message',                               'committed_date', 'no._of_branches', 'merge_commit?',
                                        'no._of_mod_files', 'dmm_unit_size', 'dmm_unit_complexity','dmm_unit_interfacing',
                                        'file_path', 'complexity','functions', 'lines_added', 'lines_removed', 'size', 'tokens',
                                        'type']]
        # Commit hash raw value is very long. Cutting the value into first ten chars 
        # Assumption is that the first ten chars rednder necessary uniqueness. May need to revisit later
        #comb_frame_refined['hash'] = comb_frame_refined['hash'].str.slice(0,10)

        # Changing the type from string to Data. Used Pacific time zone. Heard the pacific coast is beautiful
        commit_frame['committed_date'] = commit_frame['committed_date'].astype('str').apply(lambda x: pd.to_datetime(x).tz_convert('US/Pacific'))
        # Sorting the frame by committes date
        comb_frame_refined = comb_frame_refined.drop_duplicates().sort_values('committed_date', ascending=False)
        
        return comb_frame_refined

In [ ]:
se = get_latest_commits(p2[0],p2[1],p2[2])
#get_latest_commits(p6[0],p6[1],p6[2])
se[se['type']=='Buggy']

In [ ]:
# Following cells contain the code catering for K-Means clustering

In [ ]:
# Processing data for few Git Repos. Look into the above method definitions for detail
#p1 = create_components('https://github.com/R-Knowsys/elasticray.git','',localdir)
#p2 = create_components('git://localhost/grimoirelab.git','',localdir)
#p3 = create_components('git://localhost/vscode.git','',localdir)
#p4 = create_components('https://github.com/tensorflow/tensorflow.git','',localdir)    
#p5 = create_components('https://github.com/facebook/react-native.git','',localdir)
#p6 = create_components('https://github.com/kcramakrishna/Digital_Assistant_Client.git','',localdir)
#p7 = create_components('git://localhost/tensorflow.git','',localdir)
#p8 = create_components('git://localhost/Python.git','',localdir)
#p9 = create_components('git://localhost/keras.git','',localdir)
#p10 = create_components('git://localhost/requests.git','',localdir)
p11 = create_components('git://localhost/scrapy.git','',localdir)
#p12 = create_components('git://localhost/flask.git','',localdir)


# Getting the commit data as pandas frames
#p1_commits = get_latest_commits(p1[0],p1[1],p1[2])
#p2_commits = get_latest_commits(p2[0],p2[1],p2[2])
#p3_commits = get_latest_commits(p3[0],p3[1],p3[2])
#p4_commits = get_latest_commits(p4[0],p4[1],p4[2])
#p6_commits = get_latest_commits(p6[0],p6[1],p6[2])
#p7_commits = get_latest_commits(p7[0],p7[1],p7[2])
#p8_commits = get_latest_commits(p8[0],p8[1],p8[2])
#p9_commits = get_latest_commits(p9[0],p9[1],p9[2])
#p10_commits = get_latest_commits(p10[0],p10[1],p10[2])
p11_commits = get_latest_commits(p11[0],p11[1],p11[2])
#p12_commits = get_latest_commits(p12[0],p12[1],p12[2])


# Combining commit data from two Git Repos. Add as many Repos as you like
#total_commits = p2_commits.append(p4_commits)
#total_commits = p11_commits
#print(total_commits.shape)
p11_commits.to_csv('scrapy.csv')
